<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
import qcutils

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))

sr_url=qcutils.read_config_value("kafka.schema_registry.url")

topic = ''

qcutils.create_kafka_topic(topic)

# write a kafka avro producer

Define avro schemas for keys and value to describe cards in a deck

### Key
The key has a single field to contain arbitrary `Name`

### Value
A card is described by an int `Number` and a `Suit`.

**Note**: The card havo only 4 different suits

1) define the key schema

In [ ]:
key_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CardKey",
  "fields": [
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""

2) define the value schema

In [ ]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CardValue",
  "fields": [
    {
      "name": "number",
      "type": "int"
    },
    {
      "name": "suittype",
      "type": {
        "type": "enum",
        "name": "Suit",
        "symbols": ["SPADES", "HEARTS", "DIAMONDS", "CLUBS"]
      },
      "doc": "The suit of the card"
    }
  ]
}
"""

3) create a topic `<surname>-avro-cards` and send in messages such as `SPADES 2`, `HEARTS Q`, `CLUBS K`, ...

In [ ]:
key_schema = avro.loads(key_schema_str)
value_schema = avro.loads(value_schema_str)

In [ ]:
def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"name": "Two of spades"}
value = {"number": 2, "suittype": "SPADES"}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

key = {"name": "Seven of hearts"}
value = {"number": 7, "suittype": "HEARTS"}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

key = {"name": "Four of diamonds"}
value = {"number": 4, "suittype": "DIAMONDS"}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

key = {"name": "Nine of clubs"}
value = {"number": 9, "suittype": "CLUBS"}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

ap.flush(1)

## Schema Evolution

A system that reads the card data, needs the color of the card for a statistical analysis.

Modify the schema to add the card color, and send new data with the card color.

Do you need to change the topic or the consumer implementation?

In [ ]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CardEvolvedValue",
  "fields": [
    {
      "name": "number",
      "type": "int"
    },
    {
      "name": "suittype",
      "type": {
        "type": "enum",
        "name": "Suit",
        "symbols": ["SPADES", "HEARTS", "DIAMONDS", "CLUBS"]
      },
      "doc": "The suit of the card"
    },
    {
      "name": "color",
      "type": "string",
      "default": "white"
    }
  ]
}
"""

value_schema = avro.loads(value_schema_str)

def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
newap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"name": "Two of spades"}
value = {"number": 2, "suittype": "SPADES", "color":"red"}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

newap.flush(1)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.